In [ ]:
import bs4
import os

from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

global vectorstore

# embeddings
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

if os.path.exists("./chromadb_directory"):
    vectorstore = Chroma(persist_directory = "./chromadb_directory", embedding_function = embeddings)
else:
    pdf_loader = PyPDFLoader("./sample_textbook.pdf")
    documents = pdf_loader.load()

    vectorstore = Chroma(persist_directory="./chromadb_directory", embedding_function = embeddings)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 100
    )

    for i in range(0, len(documents), 100):
        batch = documents[i:i + 100]
        chunks = text_splitter.split_documents(batch)
        vectorstore.add_documents(chunks)
    
prompt = PromptTemplate (
    input_variables=["context", "question"],
    template="""
    You are an assistant for question-answering tasks. 
    Use documentation to answer the question.
    If you don't know the answer, just say you don't know.
    Be as concise as possible.
    {context}
    Question: {question}
    """
)

retriever = vectorstore.as_retriever()

llm = ChatOllama (
      model="llama3.1:8b",
      base_url="http://127.0.0.1:11434",
      temperature=0.3
      )

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("What is Kinematics?"))
print("\n")
print(rag_chain.invoke("What do dogs like to eat?"))


